# 如何流式运行可执行程序
:::info 前提条件
本指南假定您熟悉以下概念：- [聊天模型](/docs/concepts/chat_models)- [LangChain 表达式语言](/docs/concepts/lcel)- [输出解析器](/docs/concepts/output_parsers)
:::
流式传输对于让基于LLM的应用程序对终端用户具有响应感至关重要。
重要的LangChain基础组件如[聊天模型](/docs/concepts/chat_models)、[输出解析器](/docs/concepts/output_parsers)、[提示模板](/docs/concepts/prompt_templates)、[检索器](/docs/concepts/retrievers)和[智能体](/docs/concepts/agents)均实现了LangChain的[可运行接口](/docs/concepts/runnables)。
此接口提供了两种通用的流式内容处理方法：
1. 同步 `stream` 与异步 `astream`：一种**默认实现**的流式传输方式，用于从链中流式传输**最终输出**。2. 异步 `astream_events` 与异步 `astream_log`：这些方法提供了从链中流式传输**中间步骤**和**最终输出**的能力。
让我们来看看这两种方法，并尝试理解如何使用它们。
:::信息要获取LangChain中流式技术的更高级概述，请参阅[概念指南的这一部分](/docs/concepts/streaming)。:::
## 使用流
所有 `Runnable` 对象都实现了一个名为 `stream` 的同步方法，以及一个名为 `astream` 的异步变体。
这些方法旨在以分块形式流式传输最终输出，一旦每个块可用就立即生成。
只有当程序中的所有步骤都知晓如何处理**输入流**时，才能实现流式处理；即每次处理一个输入数据块，并生成相应的输出数据块。
这种处理的复杂性可能各不相同，从直接输出大语言模型生成的令牌这类简单任务，到在完整JSON生成前流式传输部分JSON结果这类更具挑战性的任务。
探索流式处理的最佳起点是大型语言模型（LLM）应用中最核心的组件——LLM模型本身！
### 大语言模型与聊天模型
大型语言模型及其聊天变体是基于LLM应用的主要瓶颈。
大型语言模型可能需要**几秒钟**才能生成对查询的完整响应。这远远慢于**约200-300毫秒**的阈值——在该响应时间内，应用程序才会让终端用户感觉反应灵敏。
使应用程序感觉更灵敏的关键策略是展示中间进度；即**逐词**流式传输模型的输出。
我们将展示使用聊天模型进行流式处理的示例。请从以下选项中选择一个：
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs
customVarName="模型"/>

In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

keys = [
    "ANTHROPIC_API_KEY",
    "OPENAI_API_KEY",
]

for key in keys:
    if key not in os.environ:
        os.environ[key] = getpass(f"Enter API Key for {key}=?")


from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


让我们从同步 `stream` API 开始：

In [2]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| sky| appears| blue| during| the| day|.|

或者，如果您在异步环境中工作，可以考虑使用异步的 `astream` API：

In [3]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| sky| appears| blue| during| the| day|.|

让我们检查其中一个数据块

In [4]:
chunks[0]

AIMessageChunk(content='The', id='run-b36bea64-5511-4d7a-b6a3-a07b3db0c8e7')

我们得到的是一个名为 `AIMessageChunk` 的返回内容。这个块（chunk）代表 `AIMessage` 的一部分。
消息块的设计是可叠加的——只需将它们简单相加，就能得到当前响应的状态！

In [5]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='The sky appears blue during', id='run-b36bea64-5511-4d7a-b6a3-a07b3db0c8e7')

### 链式结构
几乎所有LLM应用都涉及多个步骤，而不仅仅是对语言模型的一次调用。
让我们使用`LangChain表达式语言`(`LCEL`)构建一个简单的链，该链结合了提示词、模型和解析器，并验证流式传输是否正常工作。
我们将使用 [`StrOutputParser`](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) 来解析模型的输出。这是一个简单的解析器，它会从 `AIMessageChunk` 中提取 `content` 字段，从而获取模型返回的 `token`。
:::提示LCEL 是一种通过将不同LangChain原语链接在一起来定义「程序」的*声明式*方法。使用LCEL创建的链能自动实现`stream`和`astream`方法，从而支持最终输出的流式传输。实际上，基于LCEL构建的链完整实现了标准Runnable接口的全部功能。:::

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

Here|'s| a| joke| about| a| par|rot|:|

A man| goes| to| a| pet| shop| to| buy| a| par|rot|.| The| shop| owner| shows| him| two| stunning| pa|rr|ots| with| beautiful| pl|um|age|.|

"|There|'s| a| talking| par|rot| an|d a| non|-|talking| par|rot|,"| the| owner| says|.| "|The| talking| par|rot| costs| $|100|,| an|d the| non|-|talking| par|rot| is| $|20|."|

The| man| says|,| "|I|'ll| take| the| non|-|talking| par|rot| at| $|20|."|

He| pays| an|d leaves| with| the| par|rot|.| As| he|'s| walking| down| the| street|,| the| par|rot| looks| up| at| him| an|d says|,| "|You| know|,| you| really| are| a| stupi|d man|!"|

The| man| is| stun|ne|d an|d looks| at| the| par|rot| in| dis|bel|ief|.| The| par|rot| continues|,| "|Yes|,| you| got| r|ippe|d off| big| time|!| I| can| talk| just| as| well| as| that| other| par|rot|,| an|d you| only| pai|d $|20| |for| me|!"|

请注意，尽管我们在上述链的末尾使用了 `parser`，但仍然获得了流式输出。这是因为 `parser` 会对每个流式数据块单独进行处理。许多 [LCEL 基础组件](/docs/how_to#langchain-expression-language-lcel) 也支持这种转换式直通流处理，这在构建应用程序时非常方便。
自定义函数可以[设计为返回生成器](/docs/how_to/functions#streaming)，这些生成器能够对流进行操作。
某些可运行对象，例如[提示模板](/docs/how_to#prompt-templates)和[聊天模型](/docs/how_to#chat-models)，无法处理单个数据块，而是会聚合所有先前的步骤。这类可运行对象可能会中断流式处理过程。

:::noteLangChain表达式语言允许你将链的构建与其使用模式（如同步/异步、批处理/流处理等）分离开来。如果这与你的构建内容无关，你也可以通过标准的**命令式**编程方法来实现。对每个组件单独调用 `invoke`、`batch` 或 `stream`，将结果赋值给变量，然后根据需求在下游使用这些变量。
好的,我将以标准markdown格式进行翻译,只输出具体内容:

# 欢迎使用翻译助手

这是一个将英文翻译成中文的助手工具。以下是使用说明:

## 功能特点

- 支持多种文件格式翻译
- 保持原文格式和排版
- 快速准确的翻译结果
- 支持专业术语库

## 使用方法

1. 输入或粘贴需要翻译的英文文本
2. 点击"翻译"按钮
3. 获取准确的中文翻译结果

## 注意事项

- 请确保输入文本为英文
- 复杂格式可能需要人工校对
- 专业领域建议使用定制术语库

如需帮助,请联系我们: contact@transhelper.com

### 处理输入流
如果你想在生成过程中实时流式传输JSON输出该怎么办？
如果你依赖 `json.loads` 来解析不完整的 JSON，解析将会失败，因为不完整的 JSON 不是有效的 JSON。
你可能会完全不知所措，声称无法流式传输JSON数据。
原来，确实有一种方法可以实现——解析器需要操作**输入流**，并尝试将不完整的 JSON "自动补全"为有效状态。
让我们通过实际操作来观察这样一个解析器，以理解其含义。

In [7]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 67}]}
{'countries': [{'name': 'France', 'population': 67413}]}
{'countries': [{'name': 'France', 'population': 67413000}]}
{'countries': [{'name': 'France', 'population': 67413000}, {}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47351}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47351567}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47351567}, {}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 4735156

现在，让我们**打破**流式传输。我们将使用之前的示例，并在最后附加一个提取函数，用于从最终确定的JSON中提取国家名称。
:::警告在链式操作中，任何对**最终输入**而非**输入流**进行处理的步骤，都可能通过`stream`或`astream`破坏流式功能。:::
:::提示稍后，我们将讨论`astream_events` API，该接口可流式传输来自中间步骤的结果。即使链中包含仅对**最终确定输入**进行操作的步骤，此API仍会持续输出中间步骤的流式结果。好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个**markdown格式**的翻译示例。

## 主要功能

1. 保持原始文档结构
2. 准确传达原文含义
3. 符合中文表达习惯

### 注意事项

* 不添加额外说明
* 保持格式一致
* 仅输出翻译内容

> 翻译需要兼顾准确性和可读性

[这是一个链接示例](https://example.com)

```python
# 代码块也会原样保留
def example():
    print("Hello World")
```

In [8]:
from langchain_core.output_parsers import (
    JsonOutputParser,
)


# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

#### 生成器函数
让我们使用一个能够在**输入流**上操作的生成器函数来修复流式传输问题。
:::提示生成器函数（使用 `yield` 的函数）允许编写处理**输入流**的代码好的，请提供需要翻译的英文文本，我会将其转换为标准的中文Markdown格式，并保持原有的排版结构。

In [9]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

:::注意由于上述代码依赖于JSON自动补全功能，您可能会看到国家名称的部分片段（例如`Sp`和`Spain`），而这并非提取结果所期望的！
我们关注的是流式处理的概念，而非链条处理的结果。好的，请提供需要翻译的英文文本，我会将其翻译成中文并保持原有的Markdown格式。

### 非流式组件
某些内置组件（如检索器）不提供任何`流式传输`功能。如果我们尝试对它们进行`流式传输`会发生什么呢？🤨

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(page_content='harrison worked at kensho'),
  Document(page_content='harrison likes spicy food')]]

该数据流刚刚输出了该组件的最终结果。
这样是可以的 🥹！并非所有组件都必须实现流式传输——在某些情况下，流式传输要么没有必要，要么难以实现，要么根本不合逻辑。
:::提示使用非流式组件构建的LCEL链，在许多情况下仍能实现流式输出，其部分输出的流式传输将在链中最后一个非流式步骤之后开始。好的, 以下是翻译成中文的markdown格式内容:

# 欢迎使用翻译助手

## 功能特点

1. **多语言支持**: 支持多种语言的互译
2. **格式保留**: 保持原文的markdown格式不变
3. **快速响应**: 提供即时翻译服务

## 使用方法

```python
def translate(text):
    # 这里是翻译函数的示例代码
    return translated_text
```

> 注意: 请确保输入的文本是标准的markdown格式

| 功能 | 描述 |
|------|------|
| 翻译 | 将文本从一种语言转换为另一种语言 |
| 格式化 | 保持原有的文本格式 |

[点击这里](#) 了解更多信息

In [11]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)

Base|d on| the| given| context|,| Harrison| worke|d at| K|ens|ho|.|

Here| are| |3| |made| up| sentences| about| this| place|:|

1|.| K|ens|ho| was| a| cutting|-|edge| technology| company| known| for| its| innovative| solutions| in| artificial| intelligence| an|d data| analytics|.|

2|.| The| modern| office| space| at| K|ens|ho| feature|d open| floor| plans|,| collaborative| work|sp|aces|,| an|d a| vib|rant| atmosphere| that| fos|tere|d creativity| an|d team|work|.|

3|.| With| its| prime| location| in| the| heart| of| the| city|,| K|ens|ho| attracte|d top| talent| from| aroun|d the| worl|d,| creating| a| diverse| an|d dynamic| work| environment|.|

既然我们已经了解了 `stream` 和 `astream` 的运作方式，现在让我们一同探索流式事件的世界吧。🏞️

## 使用流事件
事件流是一个**测试版**API。该API可能会根据反馈进行一些调整。
:::note
本指南展示的是 `V2` 版 API，要求 langchain-core 版本 ≥ 0.2。如需兼容旧版 LangChain 的 `V1` 版 API，请参阅[此处](https://python.langchain.com/v0.1/docs/expression_language/streaming/#using-stream-events)。好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个**markdown格式**的翻译示例:

## 主要功能
1. 保持原始文档结构
2. 保留所有格式标记
3. 提供准确自然的翻译

### 注意事项
- 不添加额外说明文字
- 严格遵循原文排版
- 仅输出翻译内容

> 这是引用的翻译示例  
> 多行内容也会正确处理

`代码块`和[链接](https://example.com)等元素都会保留原格式。

表格示例:

| 项目 | 描述 |
|------|------|
| 质量 | 高标准 |
| 速度 | 快速响应 |

请提供需要翻译的英文内容。

In [ ]:
import langchain_core

langchain_core.__version__

为确保 `astream_events` API 正常运行：
* 在代码中尽可能使用 `async`（例如异步工具等）* 在定义自定义函数/可运行对象时传播回调* 在不使用LCEL的情况下使用可运行对象时，请确保在LLM上调用`.astream()`而非`.ainvoke`，以强制LLM流式传输令牌。* 如有任何功能未按预期运行，请告知我们！ :)
### 事件参考
以下是参考表格，展示了各种可运行对象可能触发的事件。

:::note当流式传输正确实现时，可运行对象的输入只有在输入流被完全消费后才会知晓。这意味着`inputs`通常只会在`end`事件中包含，而不会出现在`start`事件中。:::
| 事件                | 名称             | 数据块                          | 输入                                          | 输出                                           ||----------------------|------------------|---------------------------------|-----------------------------------------------|-------------------------------------------------|| on_chat_model_start  | [模型名称]     |                                 | \{"messages": [[系统消息, 用户消息]]\} |                                                 || on_chat_model_stream | [模型名称]     | AIMessageChunk(content="你好") |                                               |                                                 || on_chat_model_end    | [模型名称]     |                                 | \{"messages": [[系统消息, 人工消息]]\} | AI消息块(content="你好世界")           || on_llm_start         | [模型名称]     |                                 | \{'input': '你好'\}                            |                                                 || on_llm_stream        | [模型名称]     | '你好'                         |                                               |                                                 || on_llm_end           | [模型名称]     |                                 | '你好，人类！'                                |                                                 || on_chain_start       | format_docs      |                                 |                                               |                                                 || on_chain_stream      | format_docs      | "你好，世界！再见，世界！"  |                                               |                                                 || on_chain_end         | format_docs      |                                 | [Document(...)]                               | "你好，世界！再见，世界！"                  || on_tool_start        | some_tool        |                                 | \{"x": 1, "y": "2"\}                            |                                                 || on_tool_end          | some_tool        |                                 |                                               | \{"x": 1, "y": "2"\}                              || on_retriever_start   | [检索器名称] |                                 | \{"query": "你好"\}                            |                                                 || on_retriever_end     | [检索器名称]     |                                 | \{"query": "你好"\}                             | [文档(...), ..]                                || on_prompt_start      | [模板名称]       |                                 | \{"question": "你好"\}                          |                                                 || on_prompt_end        | [模板名称]       |                                 | \{"question": "你好"\}                          | 聊天提示值(消息列表: [系统消息, ...])          |

### 聊天模型
让我们首先来看看聊天模型产生的事件。

In [13]:
events = []
async for event in model.astream_events("hello"):
    events.append(event)

:::note
注意
对于 `langchain-core<0.3.37` 版本，需显式设置 `version` 参数（例如：`model.astream_events("hello", version="v2")`）。
:::

让我们来看几个开始事件和几个结束事件。

In [14]:
events[:3]

[{'event': 'on_chat_model_start',
  'data': {'input': 'hello'},
  'name': 'ChatAnthropic',
  'tags': [],
  'run_id': 'b18d016d-8b9b-49e7-a555-44db498fcf66',
  'metadata': {'ls_provider': 'anthropic',
   'ls_model_name': 'claude-3-sonnet-20240229',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 1024},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': 'b18d016d-8b9b-49e7-a555-44db498fcf66',
  'name': 'ChatAnthropic',
  'tags': [],
  'metadata': {'ls_provider': 'anthropic',
   'ls_model_name': 'claude-3-sonnet-20240229',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 1024},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-b18d016d-8b9b-49e7-a555-44db498fcf66', usage_metadata={'input_tokens': 8, 'output_tokens': 4, 'total_tokens': 12, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': 'b1

In [15]:
events[-2:]

[{'event': 'on_chat_model_stream',
  'run_id': 'b18d016d-8b9b-49e7-a555-44db498fcf66',
  'name': 'ChatAnthropic',
  'tags': [],
  'metadata': {'ls_provider': 'anthropic',
   'ls_model_name': 'claude-3-sonnet-20240229',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 1024},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b18d016d-8b9b-49e7-a555-44db498fcf66', usage_metadata={'input_tokens': 0, 'output_tokens': 12, 'total_tokens': 12, 'input_token_details': {}})},
  'parent_ids': []},
 {'event': 'on_chat_model_end',
  'data': {'output': AIMessageChunk(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b18d016d-8b9b-49e7-a555-44db498fcf66', usage_metadata={'input_tokens': 8, 'output_tokens': 16, 'total_tokens': 24, 'input_token_details': {'cache_creation': 0, 'cache_read':

### 链
让我们重新审视解析流式JSON的示例链，以探索流式事件API。

In [16]:
chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models

events = [
    event
    async for event in chain.astream_events(
        "output a list of the countries france, spain and japan and their populations in JSON format. "
        'Use a dict with an outer key of "countries" which contains a list of countries. '
        "Each country should have the key `name` and `population`",
    )
]

如果查看前几个事件，你会发现有 **3** 个不同的开始事件，而不是 **2** 个。
三个起始事件对应：
1. 链式结构（模型 + 解析器）2. 模型3. 解析器

In [18]:
events[:3]

[{'event': 'on_chain_start',
  'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': '4765006b-16e2-4b1d-a523-edd9fd64cb92',
  'metadata': {}},
 {'event': 'on_chat_model_start',
  'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`')]]}},
  'name': 'ChatAnthropic',
  'tags': ['seq:step:1'],
  'run_id': '0320c234-7b52-4a14-ae4e-5f100949e589',
  'metadata': {}},
 {'event': 'on_chat_model_stream',
  'data': {'chunk': AIMessageChunk(content='{', id='run-0320c234-7b52-4a14-ae4e-5f100949e589')},
  'run_id': '0320c

你认为如果查看最后3个事件会看到什么？中间的呢？

让我们使用这个API来输出模型和解析器的流事件。我们将忽略开始事件、结束事件以及来自链的事件。

In [19]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: '{'
Parser chunk: {}
Chat model chunk: '\n  "countries'
Chat model chunk: '": [\n    '
Parser chunk: {'countries': []}
Chat model chunk: '{\n      "'
Parser chunk: {'countries': [{}]}
Chat model chunk: 'name": "France'
Parser chunk: {'countries': [{'name': 'France'}]}
Chat model chunk: '",\n      "'
Chat model chunk: 'population": 67'
Parser chunk: {'countries': [{'name': 'France', 'population': 67}]}
Chat model chunk: '413'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413}]}
Chat model chunk: '000\n    },'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}]}
Chat model chunk: '\n    {'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}, {}]}
Chat model chunk: '\n      "name":'
...


由于模型和解析器均支持流式处理，我们可以实时看到来自这两个组件的流式事件！有点酷炫，不是吗？🦜

### 事件筛选
由于该API会产生大量事件，因此能够对事件进行过滤非常有用。
您可以通过组件`名称`、组件`标签`或组件`类型`进行筛选。
#### 按名称

In [20]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    include_names=["my_parser"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'metadata': {}, 'parent_ids': ['5a0bc625-09fd-4bdf-9932-54909a9a8c29']}
{'event': 'on_parser_stream', 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['5a0bc625-09fd-4bdf-9932-54909a9a8c29']}
{'event': 'on_parser_stream', 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['5a0bc625-09fd-4bdf-9932-54909a9a8c29']}
{'event': 'on_parser_stream', 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'name': 'my_parse

#### 按类型

In [21]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    include_types=["chat_model"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chat_model_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'model', 'tags': ['seq:step:1'], 'run_id': '156c3e40-82fb-49ff-8e41-9e998061be8c', 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-sonnet-20240229', 'ls_model_type': 'chat', 'ls_temperature': 0.0, 'ls_max_tokens': 1024}, 'parent_ids': ['7b927055-bc1b-4b50-a34c-10d3cfcb3899']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-156c3e40-82fb-49ff-8e41-9e998061be8c', usage_metadata={'input_tokens': 56, 'output_tokens': 1, 'total_tokens': 57, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})}, 'run_id': '156c3e40-82fb-49ff-8e41-9e998061be8c', 'name': 'model', 'tags': ['seq:step:1'], 'metada

#### 按标签分类
:::注意
标签会被给定可运行组件的子组件继承。
如果你正在使用标签进行筛选，请确保这是你想要的操作。好的,我会按照要求进行翻译,确保markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个**markdown格式**的翻译示例。

## 功能特点

1. 保持原始格式
2. 准确传达语义
3. 自然流畅的表达

> 翻译不仅是语言的转换,更是文化的桥梁

```python
# 代码块也会保留原格式
def hello():
    print("你好,世界!")
```

[这是一个链接示例](https://example.com)

* 列表项1
* 列表项2
* 列表项3

**注意**: 我只输出翻译后的具体内容,不包含任何额外的标记或说明。

In [22]:
chain = (model | JsonOutputParser()).with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    include_tags=["my_chain"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chain_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'RunnableSequence', 'tags': ['my_chain'], 'run_id': '58d1302e-36ce-4df7-a3cb-47cb73d57e44', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatAnthropic', 'tags': ['seq:step:1', 'my_chain'], 'run_id': '8222e8a1-d978-4f30-87fc-b2dba838774b', 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-sonnet-20240229', 'ls_model_type': 'ch

### 非流式组件
还记得某些组件因为无法处理**输入流**而导致流式传输效果不佳的情况吗？
在使用`astream`时，虽然这类组件可能会中断最终输出的流式传输，但`astream_events`仍会从支持流式传输的中间步骤生成流式事件！

In [23]:
# Function that does not support streaming.
# It operates on the finalizes inputs rather than
# operating on the input stream.
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = (
    model | JsonOutputParser() | _extract_country_names
)  # This parser only works with OpenAI right now

正如预期的那样，`astream` API 无法正常工作，因为 `_extract_country_names` 不支持流式处理。

In [24]:
async for chunk in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(chunk, flush=True)

['France', 'Spain', 'Japan']


现在，让我们通过 `astream_events` 确认是否仍能从模型和解析器中获取流式输出。

In [25]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: '{'
Parser chunk: {}
Chat model chunk: '\n  "countries'
Chat model chunk: '": [\n    '
Parser chunk: {'countries': []}
Chat model chunk: '{\n      "'
Parser chunk: {'countries': [{}]}
Chat model chunk: 'name": "France'
Parser chunk: {'countries': [{'name': 'France'}]}
Chat model chunk: '",\n      "'
Chat model chunk: 'population": 67'
Parser chunk: {'countries': [{'name': 'France', 'population': 67}]}
Chat model chunk: '413'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413}]}
Chat model chunk: '000\n    },'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}]}
Chat model chunk: '\n    {'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}, {}]}
Chat model chunk: '\n      "name":'
Chat model chunk: ' "Spain",'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain'}]}
Chat model chunk: '\n      "population":'
Chat model chunk: ' 47'
Parser chunk: {'countrie

### 回调传播
:::注意如果你在工具内部调用可运行对象，需要将回调函数传递给该可运行对象；否则不会生成任何流式事件。:::
:::note在使用 `RunnableLambdas` 或 `@chain` 装饰器时，回调函数会在后台自动传播。好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个标准的markdown格式文档示例。

## 主要功能

1. 提供高质量的翻译服务
2. 保持原始文档格式
3. 支持多种文件格式

### 使用说明

- 输入需要翻译的文本
- 选择目标语言
- 获取翻译结果

> 注意:请确保输入文本的准确性以获得最佳翻译效果

**重要提示**:本服务完全免费!

[点击这里](https://example.com)了解更多信息

In [26]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool


def reverse_word(word: str):
    return word[::-1]


reverse_word = RunnableLambda(reverse_word)


@tool
def bad_tool(word: str):
    """Custom tool that doesn't propagate callbacks."""
    return reverse_word.invoke(word)


async for event in bad_tool.astream_events("hello"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': 'ea900472-a8f7-425d-b627-facdef936ee8', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '77b01284-0515-48f4-8d7c-eb27c1882f86', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '77b01284-0515-48f4-8d7c-eb27c1882f86', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'ea900472-a8f7-425d-b627-facdef936ee8', 'name': 'bad_tool', 'tags': [], 'metadata': {}}


以下是正确传递回调的重新实现。您会注意到，现在我们也从 `reverse_word` 可运行对象获取事件了。

In [27]:
@tool
def correct_tool(word: str, callbacks):
    """A tool that correctly propagates callbacks."""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("hello"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': 'd5ea83b9-9278-49cc-9f1d-aa302d671040', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '44dafbf4-2f87-412b-ae0e-9f71713810df', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '44dafbf4-2f87-412b-ae0e-9f71713810df', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'd5ea83b9-9278-49cc-9f1d-aa302d671040', 'name': 'correct_tool', 'tags': [], 'metadata': {}}


如果你在可运行Lambda或`@chains`中调用可运行对象，那么回调将自动代表你传递。

In [28]:
from langchain_core.runnables import RunnableLambda


async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


reverse_and_double = RunnableLambda(reverse_and_double)

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '03b0e6a1-3e60-42fc-8373-1e7829198d80', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '5cf26fc8-840b-4642-98ed-623dda28707a', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '5cf26fc8-840b-4642-98ed-623dda28707a', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_chain_stream', 'data': {'chunk': '43214321'}, 'run_id': '03b0e6a1-3e60-42fc-8373-1e7829198d80', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '03b0e6a1-3e60-42fc-8373-1e7829198d80', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}


使用 `@chain` 装饰器时：

In [29]:
from langchain_core.runnables import chain


@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '1bfcaedc-f4aa-4d8e-beee-9bba6ef17008', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '64fc99f0-5d7d-442b-b4f5-4537129f67d1', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '64fc99f0-5d7d-442b-b4f5-4537129f67d1', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_chain_stream', 'data': {'chunk': '43214321'}, 'run_id': '1bfcaedc-f4aa-4d8e-beee-9bba6ef17008', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '1bfcaedc-f4aa-4d8e-beee-9bba6ef17008', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}


## 后续步骤
现在你已经学会了一些使用LangChain流式传输最终输出和内部步骤的方法。
要了解更多信息，请查看本节中的其他操作指南，或参阅[Langchain表达式语言的概念指南](/docs/concepts/lcel/)。